# TP facultatif : Préparation du jeu de données brut

Quand on se lance avec enthousiasme dans un nouveau projet de machine-learning, on pense avant tout au choix du modèle que l'on va utiliser. Cependant, et même s'il ne s'agit pas de l'étape la plus agréable du travail, un préalable indispensable est de réunir les données brutes et de les mettre en forme pour qu'elles puissent être ingérées par le modèle.

L'objectif de ce TP est de comprendre comment, à partir de différentes sources de données, on construit nos jeux de données "propres" pour ensuite construire le meilleur modèle d'apprentissage possible pour la prévision de consommation nationale.

Nos fichiers d'entrée bruts sont les suivants :
> * historique de consommation nationale (source : Eco2mix) : Yconso_2014_2018.csv
> * calendrier des jours fériés : joursFeries.csv
> * coordonnées géographiques et les poids associés aux stations Météo France : StationsMeteoRTE.csv   
> * historique d'observations et de prévisions à 24h de température : Xtemperature.csv

Et les fichiers que l'on va créer sont :
> * Xinput.csv  # Les entrées pour le modèle d'apprentissage
> * Yconso.csv  # les sorties pour le modèle d'apprentissage

## Environnement

Chargement des librairies python, et quelques éléments de configuration.

In [142]:
# Exécutez la cellule ci-dessous (par exemple avec shift-entrée)
# Si vous exécuter ce notebook depuis votre PC, il faudra peut-etre installer certaines librairies avec 
# 'pip3 install ma_librairie'
import os  # accès aux commandes système
import datetime  # structure de données pour gérer des objets calendaires
import pandas as pd  # gérer des tables de données en python
import numpy as np  # librairie d'opérations mathématiques
import zipfile # manipulation de fichiers zip
import urllib3 # téléchargement de fichier

%autosave 0

data_folder = os.path.join(os.getcwd(),"data")
data_folder = os.path.join("D:\\Users\\dordonnatvir\documents\FIFA\TP_Formation_Conso_MachineLearning","data")

print("")
print("Mon repertoire de data est : {}".format(data_folder))
print("")
print("Fichiers contenus dans ce répertoire :")
for file in os.listdir(data_folder):
    print(" - " + file)

Autosave disabled

Mon repertoire de data est : D:\Users\dordonnatvir\documents\FIFA\TP_Formation_Conso_MachineLearning\data

Fichiers contenus dans ce répertoire :
 - communes_coordonnees.csv
 - eCO2mix_RTE_Annuel-Definitif_2014.xls
 - eCO2mix_RTE_Annuel-Definitif_2014.zip
 - eCO2mix_RTE_Annuel-Definitif_2015.zip
 - eCO2mix_RTE_Annuel-Definitif_2016.zip
 - eCO2mix_RTE_Annuel-Definitif_2017.zip
 - eCO2mix_RTE_Annuel-Definitif_2018.xls
 - eCO2mix_RTE_Annuel-Definitif_2018.zip
 - eCO2mix_RTE_tempo_2017-2018.xls
 - eCO2mix_RTE_tempo_2017-2018.zip
 - joursFeries.csv
 - meteoX_T0_T24.zip
 - rstudio-export.zip
 - StationsMeteoRTE.csv
 - Xinput.csv
 - Xinput.zip
 - Xtemperature.csv
 - Xtemperature.zip
 - Yconso.csv
 - YconsoT0.csv
 - Yconso_2014_2018.csv


## Récupération des données


### Données de consommation

Dans un premier temps on importe les données de consommation réalisée à partir du fichier "YconsoT0.csv". La date et l'heure sont données dans les deux premières colonnes, et les autres colonnes correspondent aux consommations des 12 régions françaises (hors Corse) et à la consommation nationale.

Pour cela on utilise la bibliothèque **pandas** pour la manipulation de données et la fonction **read_csv**.

#### Import depuis un csv

In [143]:
# Les données du csv sont importé dans un objet de type dataframe
conso_csv = os.path.join(data_folder, "Yconso_2014_2018.csv")
conso_df = pd.read_csv(conso_csv, sep=",")

Il faut ensuite vérifier que les données sont importées correctement

In [144]:
# Afficher les dimensions et le noms des colonnes de la data frame
print(conso_df.shape)  # Nombre de lignes, nombre de colonnes

(43824, 2)


In [145]:
# Liste des colonnes de la data-frama
print(conso_df.columns)

Index(['ds', 'y'], dtype='object')


In [146]:
# Affichage des premières lignes
print(conso_df.head(5))
print(conso_df.tail(5))

                     ds      y
0  2014-01-01T00:00:00Z  64660
1  2014-01-01T01:00:00Z  61362
2  2014-01-01T02:00:00Z  60748
3  2014-01-01T03:00:00Z  58061
4  2014-01-01T04:00:00Z  54475
                         ds      y
43819  2018-12-31T19:00:00Z  67276
43820  2018-12-31T20:00:00Z  64679
43821  2018-12-31T21:00:00Z  61496
43822  2018-12-31T22:00:00Z  59901
43823  2018-12-31T23:00:00Z  63977


#### Petit détour pour gérer les dates

Le fichier Yconso_2014_2018.csv contient en particulier la colonnes 'ds'. Celle-ci contient une variable de type "string" correspondant à la date et à l'heure. 

<img src="pictures/clock.png" width=60 height=60>

Pour manipuler facilement cette variable, il faut la convertir en variable de type **datetime**. Cela permet notamment de calculer l'heure, le jour de la semaine, le mois, l'année.

In [147]:
# On appelle "ds" (dateStamp) cette nouvelle colonne
conso_df['ds'] = pd.to_datetime(conso_df['ds'])

In [148]:
conso_df.head(5)

,ds,y
0,2014-01-01 00:00:00+00:00,64660
1,2014-01-01 01:00:00+00:00,61362
2,2014-01-01 02:00:00+00:00,60748
3,2014-01-01 03:00:00+00:00,58061
4,2014-01-01 04:00:00+00:00,54475


La cellule ci-dessous a pour but d'illustrer comment utiliser ces objets.

In [149]:
# datetime vers string
noel_2017_date = datetime.date(2017, 12, 25)
noel_2017_str = datetime.datetime.strftime(noel_2017_date, format="%Y-%m-%d")
print("noel_2017_date vaut : {}, et est de type {}".format(noel_2017_date, str(type(noel_2017_date))))
print("noel_2017_str vaut : {}, et est de type {}".format(noel_2017_str, str(type(noel_2017_str))))
print("---")

# string vers datetime
starwars_day_2017_str = "2017-05-04"
starwars_day_2017_date = datetime.datetime.strptime(starwars_day_2017_str, "%Y-%m-%d")
print("starwars_day_2017_date vaut : {}, et est de type {}".format(starwars_day_2017_date, str(type(starwars_day_2017_date))))
print("D'ailleurs, c'était le " + str(starwars_day_2017_date.weekday() + 1) + " ème jour de la semaine, où 0 correspond à lundi et 6 correspond à dimanche")
print("starwars_day_2017_str vaut : {}, et est de type {}".format(starwars_day_2017_str, str(type(starwars_day_2017_str))))
print("---")

# Voyager dans le temps
saint_sylvestre_2017_date = datetime.date(2017, 12, 31)
bienvenu_en_2018_date = saint_sylvestre_2017_date + datetime.timedelta(days=1)
print("Le 31 décembre 2017 plus un jour ça donne le {}".format(bienvenu_en_2018_date))

noel_2017_date vaut : 2017-12-25, et est de type <class 'datetime.date'>
noel_2017_str vaut : 2017-12-25, et est de type <class 'str'>
---
starwars_day_2017_date vaut : 2017-05-04 00:00:00, et est de type <class 'datetime.datetime'>
D'ailleurs, c'était le 4 ème jour de la semaine, où 0 correspond à lundi et 6 correspond à dimanche
starwars_day_2017_str vaut : 2017-05-04, et est de type <class 'str'>
---
Le 31 décembre 2017 plus un jour ça donne le 2018-01-01


### Récuperation des jours fériés

Le fichier *joursFeries.csv* contient la liste des jours fériés. 

**Attention** : la granularité temporelle est différente du fichier de consommation.

On importe le fichier comme les données de consommation et on traite le champ ds.

In [150]:
jours_feries_csv = os.path.join(data_folder,"joursFeries.csv")
jours_feries_df = pd.read_csv(jours_feries_csv, sep=";")

jours_feries_df.head(5)

,ds,holiday
0,2012-12-25,Noel
1,2013-01-01,NouvelAn
2,2013-04-01,Paques
3,2013-05-01,1erMai
4,2013-05-08,8Mai


In [151]:
# Pour la première colonne, les dates sont au format "string"
# Nous allons les convertir en objet "datetime" mieux adaptés pour la manipulation de dates
print("Après import du csv, la colonne ds est de type " + str(type(jours_feries_df.ds[0])))

Après import du csv, la colonne ds est de type <class 'str'>


In [152]:
jours_feries_df.ds = pd.to_datetime(jours_feries_df.ds)
print("maintenant, la colonne ds est de type " + str(type(jours_feries_df.ds[0])))

maintenant, la colonne ds est de type <class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [153]:
jours_feries_df.head(8)

,ds,holiday
0,2012-12-25,Noel
1,2013-01-01,NouvelAn
2,2013-04-01,Paques
3,2013-05-01,1erMai
4,2013-05-08,8Mai
5,2013-05-09,Ascension
6,2013-05-20,Pentecote
7,2013-07-14,FeteNationale


**Attention**  L'heure n'apparaît pas ici mais inmplicitement, elle vaut 00:00 pour toutes les observations.
Il faut y prendre garde pour la fusion avec les données de consommation. Pour cela, on crée dans les deux jeux de données une colonne **day**, qui ne conserve que la date. Et on utilise cette colonne pour fusionner les deux jeux de données.

## Fusion des données de consommation et de jours fériés

In [154]:
conso_df['day'] = conso_df['ds'].apply(lambda x: datetime.datetime.strftime(x, format="%Y-%m-%d"))
jours_feries_df['day'] = jours_feries_df['ds'].apply(lambda x: datetime.datetime.strftime(x, format="%Y-%m-%d"))

conso_df = pd.merge(conso_df, jours_feries_df, on='day', how="left", suffixes=("", "_tmp"))

conso_df = conso_df[["ds","y", "holiday"]]
print(conso_df.loc[0:24])

                          ds      y   holiday
0  2014-01-01 00:00:00+00:00  64660  NouvelAn
1  2014-01-01 01:00:00+00:00  61362  NouvelAn
2  2014-01-01 02:00:00+00:00  60748  NouvelAn
3  2014-01-01 03:00:00+00:00  58061  NouvelAn
4  2014-01-01 04:00:00+00:00  54475  NouvelAn
5  2014-01-01 05:00:00+00:00  52534  NouvelAn
6  2014-01-01 06:00:00+00:00  52262  NouvelAn
7  2014-01-01 07:00:00+00:00  52302  NouvelAn
8  2014-01-01 08:00:00+00:00  52429  NouvelAn
9  2014-01-01 09:00:00+00:00  51663  NouvelAn
10 2014-01-01 10:00:00+00:00  52953  NouvelAn
11 2014-01-01 11:00:00+00:00  54561  NouvelAn
12 2014-01-01 12:00:00+00:00  56449  NouvelAn
13 2014-01-01 13:00:00+00:00  59253  NouvelAn
14 2014-01-01 14:00:00+00:00  56957  NouvelAn
15 2014-01-01 15:00:00+00:00  55005  NouvelAn
16 2014-01-01 16:00:00+00:00  54249  NouvelAn
17 2014-01-01 17:00:00+00:00  55189  NouvelAn
18 2014-01-01 18:00:00+00:00  59859  NouvelAn
19 2014-01-01 19:00:00+00:00  61961  NouvelAn
20 2014-01-01 20:00:00+00:00  6195

## Récupération des coordonnées géographiques des stations météo

On va charger le csv qui à chaque station météo attribue sa longitude/latitude/poids. Pour en savoir plus sur les poids :  
https://clients.rte-france.com/lang/fr/visiteurs/services/actualites.jsp?id=9482&mode=detail

**Votre mission** :
- Importez les données contenues dans le fichier csv *StationsMeteoRTE.csv* qui se situe dans data_folder vers un dataframe *stations_meteo_df*
- Regardez à quoi ces données ressemblent 

In [155]:
# Chargez les données de StationsMeteoRTE.csv vers stations_meteo_df
stations_meteo_csv = os.path.join(data_folder, "StationsMeteoRTE.csv")
stations_meteo_df = pd.read_csv(stations_meteo_csv, sep=";")

In [156]:
stations_meteo_df.head(5)

,ID,Nom,longitude,latitude,Poids
0,2,BOULOGNE-SUR-MER,1.616670,50.716670,0.010
1,5,ABBEVILLE,1.833330,50.100000,0.010
2,15,LILLE,3.058580,50.632970,0.030
3,27,CAEN,-0.359120,49.185850,0.025
4,70,REIMS,4.031696,49.258329,0.000


In [157]:
# Pour compter le nombre de stations il suffit de compter le nombre de lignes dans le data-frame
# Ceci se fait un utilisant "shape"
nb_stations = stations_meteo_df.shape[0]
print(nb_stations)

35


## Récupération du dataframe de météo

<img src="pictures/weather.png" width=60 height=60>

On va utiliser les mêmes fonctions que précédemment pour lire le fichier **'meteoX_T.csv'**, qui est situé dans data_folder et contient les historiques de température réalisée et prévue pour différentes stations Météo France.

**Attention : Les données météo sont encryptées dans un fichier zip.**  
Pour les lire vous avez besoin d'un mot de passe qui ne peut vous être donné que dans le cadre d'un travail au sein de RTE.

In [158]:
meteo_zip = os.path.join(data_folder, "Xtemperature.zip")

In [159]:
password = ""

In [160]:
# Cette étape peut être un peu longue car le fichier est volumineux

# Pour travailler avec les fichiers zip, on utilise la bibliothèque **zipfile**.
zipfile_meteo = zipfile.ZipFile(meteo_zip)
zipfile_meteo.setpassword(bytes(password,'utf-8'))
meteo_df = pd.read_csv(zipfile_meteo.open('Xtemperature.csv'),sep=",",engine='c',header=0)
meteo_df['ds'] = pd.to_datetime(meteo_df.date_cible) 
meteo_df = meteo_df.drop(columns=['date_cible'])

In [161]:
print(meteo_df.shape)  # (nb lignes , nb_colonnes)
print(meteo_df.columns)

(43577, 71)
Index(['002_0', '002_24', '005_0', '005_24', '015_0', '015_24', '027_0',
       '027_24', '070_0', '070_24', '110_0', '110_24', '120_0', '120_24',
       '130_0', '130_24', '145_0', '145_24', '149_0', '149_24', '156_0',
       '156_24', '168_0', '168_24', '180_0', '180_24', '190_0', '190_24',
       '222_0', '222_24', '240_0', '240_24', '255_0', '255_24', '260_0',
       '260_24', '280_0', '280_24', '299_0', '299_24', '434_0', '434_24',
       '460_0', '460_24', '481_0', '481_24', '497_0', '497_24', '510_0',
       '510_24', '579_0', '579_24', '588_0', '588_24', '621_0', '621_24',
       '630_0', '630_24', '643_0', '643_24', '645_0', '645_24', '650_0',
       '650_24', '675_0', '675_24', '690_0', '690_24', '747_0', '747_24',
       'ds'],
      dtype='object')


In [162]:
print(meteo_df.head(5))
print(meteo_df.tail(5))

   002_0  002_24  005_0  005_24  015_0  015_24  027_0  027_24  070_0  070_24  \
0    7.2     7.4   5.50     6.4    5.9    5.40    5.5     6.6    6.6     5.7   
1    7.1     7.1   5.50     6.2    6.5    5.20    5.5     6.6    6.8     5.8   
2    6.9     6.9   5.20     6.0    5.4    5.20    6.0     6.6    6.8     5.7   
3    7.1     6.7   5.00     5.7    5.2    5.09    5.9     6.6    6.5     5.6   
4    7.1     6.5   5.09     5.6    5.2    5.00    6.4     6.7    5.7     5.5   

   ...  645_24  650_0  650_24  675_0  675_24  690_0  690_24  747_0  747_24  \
0  ...     7.1   9.60     7.3    0.5     3.4   5.60     6.5    4.2     7.6   
1  ...     6.6   9.69     7.1    1.0     2.8   5.60     6.4    3.6     7.7   
2  ...     6.5   8.60     6.6    0.4     2.4   5.50     6.2    3.5     7.5   
3  ...     6.5   8.69     6.2    0.5     2.0   5.40     6.1    3.5     7.1   
4  ...     6.3   8.30     5.9    0.4     1.7   5.09     5.9    3.7     6.4   

                         ds  
0 2013-12-31 23:00:0

On remarque que les données s'arrête au 21/12/2018 et que pour cette date, on ne dispose que des prévisions à 24h, pas des réalisations.

## Bonus : récupération de données depuis internet

Dans le but d'automatiser un processus, nous pouvons implémenter une fonction qui va chercher les dernières données mises à disposition sur internet.  

Pour l'exemple de la prévision de consommation, il serait pertinent de fournir en entrée du modèle l'information sur le type de jour Tempo. Les clients ayant souscrit à ce type de contrat sont incités à réduire leur consommations les jours BLANC et ROUGE, aussi on peut penser que cette information permettra d'améliorer la qualité des prédictions.

### Manipulation à la main

Avant d'implémenter la version automatique, faisons une fois à la main cette manipulation.

 - Recupérez à la main le calendrier TEMPO pour 2017-2018 :
 http://www.rte-france.com/fr/eco2mix/eco2mix-telechargement
 - Le déposer dans _data&#95;folder_
 - Le dézipper
 - Regarder les données dans excel ou autre. Notez en particulier la fin du fichier, la supprimer
 
Importez ces données dans un dataframe avec 'read_excel' de la librairie pandas ou autre méthode

In [163]:
tempo_xls = os.path.join(data_folder, "eCO2mix_RTE_tempo_2017-2018.xls")
tempo_df = pd.read_csv(tempo_xls, sep="\t", encoding="ISO-8859-1")  # ce fichier est en fait un csv et non un xls...

In [164]:
print(tempo_df.head(5))

         Date Type de jour TEMPO
0  2017-09-01               BLEU
1  2017-09-02               BLEU
2  2017-09-03               BLEU
3  2017-09-04               BLEU
4  2017-09-05               BLEU


### La même chose automatisée

On récupère maintenant automatiquement les informations sur Internet à partir de l'url, sans devoir les chercher à la main soi-même.

In [165]:
def get_tempo_data(url, data_folder, tempo_xls_zip_name):
    
    tempo_xls_zip = os.path.join(data_folder, tempo_xls_zip_name)
    
    # Récupération du fichier zip depuis internet
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    http = urllib3.PoolManager()    
    with http.request('GET', url, preload_content=False) as resp, open(tempo_xls_zip, 'wb') as out_file:
        shutil.copyfileobj(resp, out_file)
        
    with zipfile.ZipFile(tempo_xls_zip, "r") as zip_file:
        zip_file.extractall(data_folder)

    # Petite vérification
    if not os.path.isfile(tempo_xls_zip):
        print("ERROR!! {} not found in {}".format("eCO2mix_RTE_tempo_2017-2018.xls", data_folder))
        raise RuntimeError("Tempo data not uploaded :-(")

    # Import de ces données dans un dataframe
    tempo_df = pd.read_csv(tempo_xls_zip, sep="\t", encoding="ISO-8859-1")
    # Suppression du disclaimer de la dernière ligne de tempo_df, par exemple avec la méthode drop d'un dataframe
    last_row = len(tempo_df.index) - 1
    tempo_df = tempo_df.drop(tempo_df.index[last_row])

    return tempo_df

On teste la fonction définie ci-dessus. Parfois pour de sombres raisons de proxy la connection au serveur peut échouer. Comme ce TP porte sur le machine-learning on ne s'acharnera pas sur cette partie en cas d'échec :-)

In [166]:
#url = "https://eco2mix.rte-france.com/curves/downloadCalendrierTempo?season=17-18"
#tempo_xls_zip_name = "eCO2mix_RTE_tempo_2017-2018.zip"

#tempo_df = get_tempo_data(url, data_folder, tempo_xls_zip_name)

#print(tempo_df)

Pour les personnes intéressées par le webscrapping, jeter un oeil du côté de <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/" title="link to google">BeautifulSoup</a>

##  Fusion des données

<img src="pictures/fusion.png" width=600 height=200>

On va maintenant construire un dataframe unique qui regroupe toutes les données nécessaire à notre modèle de prévision. On aura ici une ligne pour chaque timestamp, et dans cette ligne à la fois notre X et notre Y pour le futur modèle de machine-learning.

In [167]:
# Dans un premier temps, on fusionne la consommation et la température.
merged_df = pd.merge(conso_df, meteo_df, on = 'ds')

In [168]:
print(merged_df.shape)
print(merged_df.columns)

(43576, 73)
Index(['ds', 'y', 'holiday', '002_0', '002_24', '005_0', '005_24', '015_0',
       '015_24', '027_0', '027_24', '070_0', '070_24', '110_0', '110_24',
       '120_0', '120_24', '130_0', '130_24', '145_0', '145_24', '149_0',
       '149_24', '156_0', '156_24', '168_0', '168_24', '180_0', '180_24',
       '190_0', '190_24', '222_0', '222_24', '240_0', '240_24', '255_0',
       '255_24', '260_0', '260_24', '280_0', '280_24', '299_0', '299_24',
       '434_0', '434_24', '460_0', '460_24', '481_0', '481_24', '497_0',
       '497_24', '510_0', '510_24', '579_0', '579_24', '588_0', '588_24',
       '621_0', '621_24', '630_0', '630_24', '643_0', '643_24', '645_0',
       '645_24', '650_0', '650_24', '675_0', '675_24', '690_0', '690_24',
       '747_0', '747_24'],
      dtype='object')


### Calcul de la température France 32 villes 

On va ajouter une colonne à notre dataframe, colonne que - par expérience/expertise - on sait pouvoir être utile pour prévoir la consommation.

La température France est une moyenne pondérée de la température de 32 stations. On a donc besoin des poids de stations_meteo_df.

In [169]:
merged_df['France_0'] = np.dot(merged_df[list(merged_df.columns[merged_df.columns.str.endswith("_0")])], stations_meteo_df['Poids'])
merged_df['France_24'] = np.dot(merged_df[list(merged_df.columns[merged_df.columns.str.endswith("_24")])], stations_meteo_df['Poids'])

In [170]:
print(merged_df.shape)
print(merged_df.columns)

(43576, 75)
Index(['ds', 'y', 'holiday', '002_0', '002_24', '005_0', '005_24', '015_0',
       '015_24', '027_0', '027_24', '070_0', '070_24', '110_0', '110_24',
       '120_0', '120_24', '130_0', '130_24', '145_0', '145_24', '149_0',
       '149_24', '156_0', '156_24', '168_0', '168_24', '180_0', '180_24',
       '190_0', '190_24', '222_0', '222_24', '240_0', '240_24', '255_0',
       '255_24', '260_0', '260_24', '280_0', '280_24', '299_0', '299_24',
       '434_0', '434_24', '460_0', '460_24', '481_0', '481_24', '497_0',
       '497_24', '510_0', '510_24', '579_0', '579_24', '588_0', '588_24',
       '621_0', '621_24', '630_0', '630_24', '643_0', '643_24', '645_0',
       '645_24', '650_0', '650_24', '675_0', '675_24', '690_0', '690_24',
       '747_0', '747_24', 'France_0', 'France_24'],
      dtype='object')


## Jeu de données complet 

Prenons quelques instants pour regarder les données que l'on a pour l'instant :

In [171]:
merged_df

,ds,y,holiday,002_0,002_24,005_0,005_24,015_0,015_24,027_0,...,650_0,650_24,675_0,675_24,690_0,690_24,747_0,747_24,France_0,France_24
0,2014-01-01 00:00:00+00:00,64660,NouvelAn,7.10,7.10,5.50,6.20,6.50,5.20,5.50,...,9.69,7.10,1.00,2.80,5.60,6.40,3.60,7.70,6.465390,5.994490
1,2014-01-01 01:00:00+00:00,61362,NouvelAn,6.90,6.90,5.20,6.00,5.40,5.20,6.00,...,8.60,6.60,0.40,2.40,5.50,6.20,3.50,7.50,6.334150,5.932260
2,2014-01-01 02:00:00+00:00,60748,NouvelAn,7.10,6.70,5.00,5.70,5.20,5.09,5.90,...,8.69,6.20,0.50,2.00,5.40,6.10,3.50,7.10,6.261400,5.907900
3,2014-01-01 03:00:00+00:00,58061,NouvelAn,7.10,6.50,5.09,5.60,5.20,5.00,6.40,...,8.30,5.90,0.40,1.70,5.09,5.90,3.70,6.40,6.305620,5.879230
4,2014-01-01 04:00:00+00:00,54475,NouvelAn,6.90,6.50,4.90,5.70,5.30,4.90,6.80,...,8.60,5.90,-0.10,1.90,4.70,5.80,3.60,5.60,6.125880,5.843200
5,2014-01-01 05:00:00+00:00,52534,NouvelAn,7.10,6.40,5.09,5.40,5.30,5.40,6.90,...,8.60,6.30,0.40,1.80,4.70,5.40,3.50,4.90,6.125880,5.432510
6,2014-01-01 06:00:00+00:00,52262,NouvelAn,6.10,6.40,5.30,5.50,5.40,5.09,7.10,...,8.19,6.20,0.20,1.60,4.70,5.30,2.60,4.59,6.123120,5.378890
7,2014-01-01 07:00:00+00:00,52302,NouvelAn,6.20,6.70,5.40,5.80,5.90,4.59,7.40,...,8.40,7.00,-0.20,2.10,4.80,5.90,1.30,4.40,6.102730,5.649030
8,2014-01-01 08:00:00+00:00,52429,NouvelAn,6.00,7.00,5.80,6.10,5.90,4.59,7.90,...,7.80,7.70,0.00,3.20,5.30,6.60,1.90,4.80,6.369100,5.975010
9,2014-01-01 09:00:00+00:00,51663,NouvelAn,6.20,7.30,6.80,6.50,6.40,4.90,8.10,...,9.60,8.80,3.20,5.00,6.20,7.70,6.10,5.70,7.350010,6.581480


In [172]:
merged_df.describe()

,y,002_0,002_24,005_0,005_24,015_0,015_24,027_0,027_24,070_0,...,650_0,650_24,675_0,675_24,690_0,690_24,747_0,747_24,France_0,France_24
count,43576.000000,43551.000000,43575.000000,43551.000000,43575.000000,43551.000000,43575.000000,43551.000000,43575.000000,43551.000000,...,43551.000000,43575.000000,43551.000000,43575.000000,43551.000000,43575.000000,43551.000000,43575.000000,43551.000000,43575.000000
mean,54046.783826,11.579542,11.729742,11.406616,11.499077,11.659079,11.629891,11.646468,11.690013,11.358906,...,16.389517,16.385763,15.935210,15.945110,16.801673,16.756448,16.554677,16.540928,12.991477,12.989294
std,11828.733740,5.316638,5.179404,6.136379,5.833656,6.735357,6.648256,5.867294,5.716437,7.419970,...,7.400202,7.272001,8.155727,7.958932,6.302711,6.139574,6.942082,6.671904,6.737861,6.727910
min,29477.000000,-6.900000,-5.300000,-7.500000,-5.800000,-7.500000,-6.500000,-8.100000,-6.300000,-11.100000,...,-5.090000,-2.600000,-7.100000,-3.100000,-1.700000,0.500000,-3.900000,-1.000000,-6.947230,-5.245000
25%,45076.000000,7.700000,7.800000,6.900000,7.200000,6.600000,6.600000,7.500000,7.600000,5.900000,...,10.700000,10.800000,9.800000,9.800000,11.800000,11.700000,11.200000,11.200000,7.811208,7.784678
50%,52473.000000,11.500000,11.700000,11.300000,11.400000,11.400000,11.400000,11.400000,11.400000,10.900000,...,16.200000,16.000000,15.500000,15.300000,16.500000,16.390000,16.300000,16.200000,12.503000,12.484950
75%,62233.000000,15.600000,15.800000,15.700000,15.600000,16.390000,16.300000,15.600000,15.600000,16.500000,...,21.900000,21.900000,21.600000,21.500000,22.100000,22.000000,21.700000,21.600000,17.851173,17.880108
max,95987.000000,35.400000,33.700000,36.200000,34.500000,36.900000,36.000000,35.900000,35.400000,36.700000,...,38.900000,37.600000,41.600000,40.100000,36.500000,35.000000,38.200000,37.700000,33.449175,34.020680


In [173]:
## Utilisation de la température observée 24h avant

In [174]:
merged_df[list(merged_df.columns[merged_df.columns.str.endswith("_0")])] = merged_df[list(merged_df.columns[merged_df.columns.str.endswith("_0")])].shift(24)
new_columns = [column.replace("France_0", "Th_real_24h_avant").replace("France_24", "Th_prev") for column in merged_df.columns]
print(new_columns)

merged_df.columns = new_columns

['ds', 'y', 'holiday', '002_0', '002_24', '005_0', '005_24', '015_0', '015_24', '027_0', '027_24', '070_0', '070_24', '110_0', '110_24', '120_0', '120_24', '130_0', '130_24', '145_0', '145_24', '149_0', '149_24', '156_0', '156_24', '168_0', '168_24', '180_0', '180_24', '190_0', '190_24', '222_0', '222_24', '240_0', '240_24', '255_0', '255_24', '260_0', '260_24', '280_0', '280_24', '299_0', '299_24', '434_0', '434_24', '460_0', '460_24', '481_0', '481_24', '497_0', '497_24', '510_0', '510_24', '579_0', '579_24', '588_0', '588_24', '621_0', '621_24', '630_0', '630_24', '643_0', '643_24', '645_0', '645_24', '650_0', '650_24', '675_0', '675_24', '690_0', '690_24', '747_0', '747_24', 'Th_real_24h_avant', 'Th_prev']


## Suppression des NaN

On souhaite avoir un jeu de données complet : pas de trous dans les dates et pour chaque point horaire, toutes les informations disponibles (consommation, température réalisée et prévue)
On affiche donc les dates pour lesquelles il manque des informations et on conservera un intervalle ne contenant pas ces dates.

NB : Attention il est normal que la colonne "_holiday_" comporte des NaN

In [175]:
mask = merged_df.drop(columns='holiday').isnull().any(axis=1)
merged_df[mask].ds

0     2014-01-01 00:00:00+00:00
1     2014-01-01 01:00:00+00:00
2     2014-01-01 02:00:00+00:00
3     2014-01-01 03:00:00+00:00
4     2014-01-01 04:00:00+00:00
5     2014-01-01 05:00:00+00:00
6     2014-01-01 06:00:00+00:00
7     2014-01-01 07:00:00+00:00
8     2014-01-01 08:00:00+00:00
9     2014-01-01 09:00:00+00:00
10    2014-01-01 10:00:00+00:00
11    2014-01-01 11:00:00+00:00
12    2014-01-01 12:00:00+00:00
13    2014-01-01 13:00:00+00:00
14    2014-01-01 14:00:00+00:00
15    2014-01-01 15:00:00+00:00
16    2014-01-01 16:00:00+00:00
17    2014-01-01 17:00:00+00:00
18    2014-01-01 18:00:00+00:00
19    2014-01-01 19:00:00+00:00
20    2014-01-01 20:00:00+00:00
21    2014-01-01 21:00:00+00:00
22    2014-01-01 22:00:00+00:00
23    2014-01-01 23:00:00+00:00
165   2014-01-07 21:00:00+00:00
Name: ds, dtype: datetime64[ns, UTC]

In [176]:
import pytz
merged_df = merged_df[(merged_df.ds >= pytz.utc.localize(datetime.datetime(year=2014, month=1, day=8))) & (merged_df.ds < pytz.utc.localize(datetime.datetime(year=2018, month=12, day=20)))]
mask = merged_df.drop(columns='holiday').isnull().any(axis=1)
merged_df[mask].ds

Series([], Name: ds, dtype: datetime64[ns, UTC])

Pour chaque point horaire, on a :
* la consommation réalisée (au niveau national)
* la température réalisée 24h avant pour chaque station météo, ainsi qu'une valeur représentative de la température moyenne à l'échelle nationale
* la prévision de température à 24 heures **pour ce point horaire** pour chaque station météo, ainsi qu'une prévision de la température moyenne France
* l'information si le point horaire appartient à un jour férié ou non (et si oui lequel)   

## Sauvegarde du fichier 

Tout d'abord on sépare les données en deux : 
- le vecteur de consommation à prévoir : y_conso
- La matrice des variables explicatives : X_input

Sachant que plus tard notre modèle aura pour mission d'établir une correspondance _f_ telle que l'on ait du mieux possible une relation *y = f(X)*.

In [177]:
y_conso = merged_df[['ds', 'y']]


In [178]:
X_input = merged_df.drop(['y'], axis=1)

In [179]:
y_conso.to_csv( os.path.join(data_folder,"Yconso.csv"), index = False)
X_input.to_csv(os.path.join(data_folder,"Xinput.csv"), index = False)

Et enfin on zip Xinput.csv avec un mot de passe.  
Depuis un terminal :

> zip -e Xinput.zip Xinput.csv